# Load the Libraries

In [1]:
# Importing Required Libraries
import pandas as pd
import numpy as np

# Load the DataSet

In [2]:
# Reading File - ratings :
Required_Columns = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
Ratings = pd.read_csv('ML_100K/u.data', sep='\t', names = Required_Columns, encoding='latin-1')
Ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
Unique_Users = Ratings.user_id.unique().shape[0]
print("No. of Unique Users:", Unique_Users)

Unique_Movies = Ratings.movie_id.unique().shape[0]
print("No. of Unique Movies:", Unique_Movies)

No. of Unique Users: 943
No. of Unique Movies: 1682


# Create Pivot Table for Users and Movies Based on Ratings

In [4]:
Data_Matrix = Ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
Data_Matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Data Pre-processing the Dataset by Replacing the NaN with 0
Data_Matrix_Preprocessed = Data_Matrix.replace(np.nan,0)
Data_Matrix_Preprocessed.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Find Cosine Similatity for Users and Items

In [6]:
from sklearn.metrics.pairwise import pairwise_distances 
User_Similarity = pairwise_distances(Data_Matrix_Preprocessed, metric='cosine')
Item_Similarity = pairwise_distances(Data_Matrix_Preprocessed.T, metric='cosine')

User_Similarity

array([[1.33226763e-15, 8.33069016e-01, 9.52540457e-01, ...,
        8.51383057e-01, 8.20492117e-01, 6.01825261e-01],
       [8.33069016e-01, 1.11022302e-16, 8.89408675e-01, ...,
        8.38515222e-01, 8.27732187e-01, 8.94202122e-01],
       [9.52540457e-01, 8.89408675e-01, 0.00000000e+00, ...,
        8.98757435e-01, 8.66583851e-01, 9.73444131e-01],
       ...,
       [8.51383057e-01, 8.38515222e-01, 8.98757435e-01, ...,
        1.11022302e-16, 8.98358201e-01, 9.04880419e-01],
       [8.20492117e-01, 8.27732187e-01, 8.66583851e-01, ...,
        8.98358201e-01, 0.00000000e+00, 8.17535338e-01],
       [6.01825261e-01, 8.94202122e-01, 9.73444131e-01, ...,
        9.04880419e-01, 8.17535338e-01, 0.00000000e+00]], shape=(943, 943))

# Using Formula for Users and Items To Calcuate the Score Value

In [7]:
def Predict_Score_Value(Ratings, Similarity, type='user'):
    if type == 'user':
        # Calculate Each User's Average Ratings
        Mean_User_Rating = Ratings.mean(axis=1).values # (Convert from Panda Series To NumPy Array)
        
        # Getting a Centered Value of Each User’s Ratings by Subtracting their Mean.(np.newaxis Used to Maintain Same Format as Mean_User_Rating)
        Rating_Difference = (Ratings - Mean_User_Rating[:, np.newaxis])
        
        # Score_Value = Reshapes the Vector to a Column + How Much Similar Users Deviate from their Mean / Divided by Total Similarity for Normalization.
        Score_Value = Mean_User_Rating[:, np.newaxis] + Similarity.dot(Rating_Difference) / np.array([np.abs(Similarity).sum(axis=1)]).T
    elif type == 'item':
        
        Score_Value = Ratings.dot(Similarity) / np.array([np.abs(Similarity).sum(axis=1)])
    return Score_Value

In [8]:
# Prediction Table
User_Prediction = Predict_Score_Value(Data_Matrix_Preprocessed, User_Similarity, type='user')
Item_Prediction = Predict_Score_Value(Data_Matrix_Preprocessed, Item_Similarity, type='item')

User_Prediction

array([[ 2.06532606,  0.73430275,  0.62992381, ...,  0.39359041,
         0.39304874,  0.3927712 ],
       [ 1.76308836,  0.38404019,  0.19617889, ..., -0.08837789,
        -0.0869183 , -0.08671183],
       [ 1.79590398,  0.32904733,  0.15882885, ..., -0.13699223,
        -0.13496852, -0.13476488],
       ...,
       [ 1.59151513,  0.27526889,  0.10219534, ..., -0.16735162,
        -0.16657451, -0.16641377],
       [ 1.81036267,  0.40479877,  0.27545013, ..., -0.00907358,
        -0.00846587, -0.00804858],
       [ 1.8384313 ,  0.47964837,  0.38496292, ...,  0.14686675,
         0.14629808,  0.14641455]], shape=(943, 1682))

### As Per the User Based Filtering, First We Have to Find the Similarity Between Input Users and Others

In [9]:
#1. Select input user
Input_User = 34

In [10]:
#2. Convert the User_Similarity_Table into DataFrame
User_Similarity_Table = pd.DataFrame(User_Similarity)
User_Similarity_Table.head()

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.332268e-15,8.330690e-01,0.952540,9.356422e-01,0.621525,0.569761,0.559633,0.680928,0.921862,0.623456,...,0.630473,0.880518,0.725124,0.810295,0.802674,0.881905,0.685928,0.851383,0.820492,0.601825
1,8.330690e-01,1.110223e-16,0.889409,8.218788e-01,0.927021,0.754157,0.892672,0.896656,0.838952,0.840138,...,0.843014,0.692058,0.641211,0.575954,0.680111,0.771417,0.773210,0.838515,0.827732,0.894202
2,9.525405e-01,8.894087e-01,0.000000,6.558493e-01,0.978755,0.927585,0.933863,0.916940,0.938960,0.934849,...,0.968125,0.957247,0.836171,0.930962,0.875755,0.973729,0.838110,0.898757,0.866584,0.973444
3,9.356422e-01,8.218788e-01,0.655849,1.110223e-16,0.968196,0.931956,0.908770,0.811940,0.898716,0.939141,...,0.947893,0.963216,0.866885,0.806529,0.853942,0.969862,0.803142,0.847959,0.829914,0.941248
4,6.215248e-01,9.270210e-01,0.978755,9.681958e-01,0.000000,0.762714,0.626400,0.751070,0.943153,0.798573,...,0.661206,0.919420,0.905076,0.920221,0.851393,0.928541,0.760045,0.860405,0.847503,0.686059


In [11]:
#3. Find Similarity User for Input_User = 34 Using Cosine Table
Similar_Input_User = User_Similarity_Table[Input_User].sort_values(ascending=True).head(5).index
Similar_Input_User

Index([34, 450, 852, 811, 509], dtype='int64')

In [12]:
#4.Convert in to list
Similar_Input_User = list(Similar_Input_User)
len(Similar_Input_User)

5

In [13]:
#5. Using Similar_User_Movieid_List, Select Movieid from Ratings Table
Similar_User_Movieid_List = []

for Similar_User in Similar_Input_User:
    Similar_User_Movieid = list(Ratings[Ratings['user_id']==Similar_User]['movie_id'])
    Similar_User_Movieid_List.append(Similar_User_Movieid)

len(Similar_User_Movieid_List)

5

In [14]:
# List of Lists Will be Visible Here
#Similar_User_Movieid_List

In [15]:
#6.Convert all the List of Lists as Single List
import itertools
Similar_User_Movieid_Single_List = list(itertools.chain.from_iterable(Similar_User_Movieid_List))
len(Similar_User_Movieid_Single_List)

663

In [16]:
#7. Unique Movieid from the Single List
Unique_Movieid_Similar_User = set(Similar_User_Movieid_Single_List)
len(Unique_Movieid_Similar_User)

590

In [17]:
#8. Input User Watched movie_list
Input_User_Watched_Movieid = list(Ratings[Ratings['user_id']==Input_User]['movie_id'].values)
Input_User_Watched_Movieid

[np.int64(312),
 np.int64(242),
 np.int64(690),
 np.int64(310),
 np.int64(259),
 np.int64(299),
 np.int64(245),
 np.int64(332),
 np.int64(329),
 np.int64(286),
 np.int64(1024),
 np.int64(324),
 np.int64(294),
 np.int64(292),
 np.int64(990),
 np.int64(289),
 np.int64(898),
 np.int64(899),
 np.int64(288),
 np.int64(991)]

In [18]:
#9. Create a list which should have recom movieid to the input user
Recommend = []
for Per_Id in Unique_Movieid_Similar_User:
    if(Per_Id in Input_User_Watched_Movieid):
        pass
    else:
        Recommend.append(Per_Id)

len(Recommend)

570

In [19]:
#sorted(Recommend)

In [20]:
# Cross Checking 
#sorted(Unique_Movieid_Similar_User)

In [21]:
# Cross Checking
#sorted(Input_User_Watched_Movieid)

In [22]:
# Checking the Common Movie List
list(set(Unique_Movieid_Similar_User) & set(Input_User_Watched_Movieid))

[np.int64(1024),
 np.int64(898),
 np.int64(259),
 np.int64(899),
 np.int64(286),
 np.int64(288),
 np.int64(289),
 np.int64(292),
 np.int64(294),
 np.int64(299),
 np.int64(690),
 np.int64(310),
 np.int64(312),
 np.int64(324),
 np.int64(329),
 np.int64(332),
 np.int64(990),
 np.int64(991),
 np.int64(242),
 np.int64(245)]

In [23]:
User_Prediction = pd.DataFrame(User_Prediction)
User_Prediction

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.065326,0.734303,0.629924,1.010669,0.640686,0.476150,1.784569,1.163032,1.513350,0.704478,...,0.394041,0.394434,0.393981,0.392972,0.393344,0.392272,0.394909,0.393590,0.393049,0.392771
1,1.763088,0.384040,0.196179,0.731538,0.225643,0.003892,1.493597,0.876153,1.108467,0.261991,...,-0.086942,-0.085491,-0.087137,-0.088158,-0.087298,-0.089288,-0.087468,-0.088378,-0.086918,-0.086712
2,1.795904,0.329047,0.158829,0.684154,0.173277,-0.035621,1.488230,0.835769,1.135426,0.236383,...,-0.134795,-0.133537,-0.135543,-0.136438,-0.135041,-0.137611,-0.136374,-0.136992,-0.134969,-0.134765
3,1.729951,0.293913,0.127741,0.644932,0.142143,-0.062261,1.437010,0.796249,1.096663,0.211789,...,-0.161413,-0.160220,-0.161542,-0.162586,-0.161634,-0.163877,-0.162283,-0.163080,-0.161442,-0.161248
4,1.796651,0.454474,0.354422,0.763130,0.359539,0.195987,1.547370,0.908904,1.292027,0.437954,...,0.101762,0.102405,0.101923,0.100839,0.101711,0.099951,0.102515,0.101233,0.101075,0.101201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1.676950,0.346339,0.177518,0.689906,0.199740,0.003297,1.429565,0.830905,1.070986,0.262183,...,-0.092434,-0.091197,-0.092851,-0.093801,-0.092953,-0.094539,-0.092217,-0.093378,-0.092686,-0.092423
939,1.822346,0.419125,0.286430,0.715605,0.294442,0.106633,1.514591,0.853050,1.195304,0.359260,...,0.014060,0.014688,0.014123,0.013060,0.013669,0.011978,0.014065,0.013021,0.013639,0.013796
940,1.591515,0.275269,0.102195,0.624383,0.133762,-0.069553,1.320734,0.765529,1.035088,0.192697,...,-0.166179,-0.164981,-0.166278,-0.167392,-0.166679,-0.168486,-0.166217,-0.167352,-0.166575,-0.166414
941,1.810363,0.404799,0.275450,0.726616,0.281316,0.087068,1.550310,0.850057,1.205745,0.342987,...,-0.008362,-0.007757,-0.008225,-0.009218,-0.008232,-0.010138,-0.008009,-0.009074,-0.008466,-0.008049


In [24]:
User_Prediction_Transpose = User_Prediction.T
User_Prediction_Transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,2.065326,1.763088,1.795904,1.729951,1.796651,1.996889,2.466055,1.693486,1.681165,2.021688,...,1.924844,1.619096,1.891812,1.696368,1.717658,1.676950,1.822346,1.591515,1.810363,1.838431
1,0.734303,0.384040,0.329047,0.293913,0.454474,0.651172,1.099186,0.296285,0.279528,0.650911,...,0.552456,0.311680,0.550513,0.322611,0.431368,0.346339,0.419125,0.275269,0.404799,0.479648
2,0.629924,0.196179,0.158829,0.127741,0.354422,0.528276,1.016489,0.195398,0.121887,0.530669,...,0.451194,0.140613,0.354913,0.144344,0.247851,0.177518,0.286430,0.102195,0.275450,0.384963
3,1.010669,0.731538,0.684154,0.644932,0.763130,0.921054,1.362861,0.612075,0.616094,0.916215,...,0.834176,0.661829,0.893214,0.662304,0.783208,0.689906,0.715605,0.624383,0.726616,0.780521
4,0.640686,0.225643,0.173277,0.142143,0.359539,0.537388,1.006458,0.198638,0.136368,0.531720,...,0.455212,0.166992,0.398759,0.167841,0.283796,0.199740,0.294442,0.133762,0.281316,0.388442


In [25]:
User_Predicted_Ratings = User_Prediction_Transpose[34]
User_Predicted_Ratings

0       1.740469
1       0.283366
2       0.114987
3       0.645096
4       0.127844
          ...   
1677   -0.183977
1678   -0.182367
1679   -0.183172
1680   -0.181400
1681   -0.181306
Name: 34, Length: 1682, dtype: float64

In [26]:
User_Ratings_Dataframe = pd.DataFrame(User_Predicted_Ratings)
User_Ratings_Dataframe

,34
0,1.740469
1,0.283366
2,0.114987
3,0.645096
4,0.127844
...,...
1677,-0.183977
1678,-0.182367
1679,-0.183172
1680,-0.181400


In [27]:
User_Ratings_Dataframe_Transposed = User_Ratings_Dataframe.T
User_Ratings_Dataframe_Transposed

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
34,1.740469,0.283366,0.114987,0.645096,0.127844,-0.081329,1.448638,0.794313,1.08901,0.194232,...,-0.181256,-0.180156,-0.181837,-0.182819,-0.181432,-0.183977,-0.182367,-0.183172,-0.1814,-0.181306


In [28]:
# Cross Verification for Random User With Random Ratings
User_Ratings_Dataframe_Transposed[3].values >=0.4

array([ True])

In [29]:
# Select Hightest Rated Movie From Recommend List which Would be Liked by Input User, Based on User prediction
Highest_Rated=[]

Input_User_Predicted = pd.DataFrame(User_Prediction_Transpose[Input_User])
Input_User_Predicted_Transposed = Input_User_Predicted.T
for Movie in Recommend:
    Predicted_Rating = Input_User_Predicted_Transposed[Movie].values
    if(Predicted_Rating>=1):
        Highest_Rated.append(Movie)

In [30]:
len(Highest_Rated)

27

In [31]:
# Now We can Give Movieid to Respective Movie List
Movie_Info_Columns = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
                        'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 
                      'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']

Info = pd.read_csv('ML_100K/u.item', sep='|', names = Movie_Info_Columns, encoding='latin-1')

In [32]:
#Creating Movie List based on Recommend Movieid
Movie_Title = []

for MovieId in Highest_Rated:
    Movie = Info[Info['movie id']==MovieId]['movie title'].values
    Movie_Title.append(Movie)

In [33]:
Movie_Title

[array(['Seven (Se7en) (1995)'], dtype=object),
 array(['I.Q. (1994)'], dtype=object),
 array(['Santa Clause, The (1994)'], dtype=object),
 array(['Free Willy (1993)'], dtype=object),
 array(['Sleepless in Seattle (1993)'], dtype=object),
 array(['Aladdin (1992)'], dtype=object),
 array(['Dances with Wolves (1990)'], dtype=object),
 array(['Snow White and the Seven Dwarfs (1937)'], dtype=object),
 array(['Spitfire Grill, The (1996)'], dtype=object),
 array(['Private Benjamin (1980)'], dtype=object),
 array(['Empire Strikes Back, The (1980)'], dtype=object),
 array(['Princess Bride, The (1987)'], dtype=object),
 array(['Apocalypse Now (1979)'], dtype=object),
 array(['GoodFellas (1990)'], dtype=object),
 array(['Henry V (1989)'], dtype=object),
 array(['Sting, The (1973)'], dtype=object),
 array(['Unforgiven (1992)'], dtype=object),
 array(['Field of Dreams (1989)'], dtype=object),
 array(['Breaking the Waves (1996)'], dtype=object),
 array(['Men in Black (1997)'], dtype=object),
 array

In [34]:
#Converting into Pure list
Movie_Title_List = []

for Movie in Movie_Title:
    print(Movie)
    MovieName = list(Movie)
    Movie_Title_List.append(MovieName)

['Seven (Se7en) (1995)']
['I.Q. (1994)']
['Santa Clause, The (1994)']
['Free Willy (1993)']
['Sleepless in Seattle (1993)']
['Aladdin (1992)']
['Dances with Wolves (1990)']
['Snow White and the Seven Dwarfs (1937)']
['Spitfire Grill, The (1996)']
['Private Benjamin (1980)']
['Empire Strikes Back, The (1980)']
['Princess Bride, The (1987)']
['Apocalypse Now (1979)']
['GoodFellas (1990)']
['Henry V (1989)']
['Sting, The (1973)']
['Unforgiven (1992)']
['Field of Dreams (1989)']
['Breaking the Waves (1996)']
['Men in Black (1997)']
['Chasing Amy (1997)']
['Sense and Sensibility (1995)']
["Marvin's Room (1996)"]
['In & Out (1997)']
['Client, The (1994)']
['Aladdin and the King of Thieves (1996)']
['Some Like It Hot (1959)']


In [35]:
#Converting into Whole List
import itertools
Final_Recommend_Movie = list(itertools.chain.from_iterable(Movie_Title_List))
Final_Recommend_Movie

['Seven (Se7en) (1995)',
 'I.Q. (1994)',
 'Santa Clause, The (1994)',
 'Free Willy (1993)',
 'Sleepless in Seattle (1993)',
 'Aladdin (1992)',
 'Dances with Wolves (1990)',
 'Snow White and the Seven Dwarfs (1937)',
 'Spitfire Grill, The (1996)',
 'Private Benjamin (1980)',
 'Empire Strikes Back, The (1980)',
 'Princess Bride, The (1987)',
 'Apocalypse Now (1979)',
 'GoodFellas (1990)',
 'Henry V (1989)',
 'Sting, The (1973)',
 'Unforgiven (1992)',
 'Field of Dreams (1989)',
 'Breaking the Waves (1996)',
 'Men in Black (1997)',
 'Chasing Amy (1997)',
 'Sense and Sensibility (1995)',
 "Marvin's Room (1996)",
 'In & Out (1997)',
 'Client, The (1994)',
 'Aladdin and the King of Thieves (1996)',
 'Some Like It Hot (1959)']

In [36]:
# Checking the Common Movie List
list(set(Recommend) & set(Input_User_Watched_Movieid))

[]

In [39]:
def User_Based(Input_User, User_Similarity, User_Predictions, Similar_User_Count, Rating_Threshold):
    
    #2. Convert the User_Similarity_Table into DataFrame
    User_Similarity_Table = pd.DataFrame(User_Similarity)
    
    #3. Find Similarity User for Input_User = Similar_User_Count Using Cosine Table
    Similar_Input_User = User_Similarity_Table[Input_User].sort_values(ascending=True).head(Similar_User_Count).index

    #4.Convert in to list
    Similar_Input_User = list(Similar_Input_User)
    
    #5. Using Similar_User_Movieid_List, Select Movieid from Ratings Table
    Similar_User_Movieid_List = []

    for Similar_User in Similar_Input_User:
        Similar_User_Movieid = list(Ratings[Ratings['user_id']==Similar_User]['movie_id'])
        Similar_User_Movieid_List.append(Similar_User_Movieid)
    
    #6.Convert all the List of Lists as Single List
    import itertools
    Similar_User_Movieid_Single_List = list(itertools.chain.from_iterable(Similar_User_Movieid_List))
    
    #7. Unique Movieid from the Single List
    Unique_Movieid_Similar_User = set(Similar_User_Movieid_Single_List)
    
    #8. Input User Watched movie_list
    Input_User_Watched_Movieid = list(Ratings[Ratings['user_id']==Input_User]['movie_id'].values)

    #9. Create a list which should have recom movieid to the input user
    Recommend = []
    
    for Per_Id in Unique_Movieid_Similar_User:
        if(Per_Id in Input_User_Watched_Movieid):
            pass
        else:
            Recommend.append(Per_Id)

    User_Ratings_Dataframe = pd.DataFrame(User_Predicted_Ratings)
    User_Ratings_Dataframe_Transposed = User_Ratings_Dataframe.T
    
    Input_User_Predicted = pd.DataFrame(User_Prediction_Transpose[Input_User])
    Input_User_Predicted_Transposed = Input_User_Predicted.T

    
    # Select Hightest Rated Movie From Recommend List which Would be Liked by Input User, Based on User prediction
    Highest_Rated=[]

    for Movie in Recommend:
        Predicted_Rating = Input_User_Predicted_Transposed[Movie].values
        if(Predicted_Rating >= Rating_Threshold):
            Highest_Rated.append(Movie)
    
    # Now We can Give Movieid to Respective Movie List
    Movie_Info_Columns = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
                            'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 
                          'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']
    
    Info = pd.read_csv('ML_100K/u.item', sep='|', names = Movie_Info_Columns, encoding='latin-1')
        
    
    #Creating Movie List based on Recommend Movieid
    Movie_Title = []

    for MovieId in Highest_Rated:
        Movie = Info[Info['movie id']==MovieId]['movie title'].values
        Movie_Title.append(Movie)
    
    #Converting into Pure list
    Movie_Title_List = []
    
    for Movie in Movie_Title:
        print(Movie)
        MovieName = list(Movie)
        Movie_Title_List.append(MovieName)

    #Converting into Whole List
    import itertools
    Final_Recommend_Movie = list(itertools.chain.from_iterable(Movie_Title_List))
    print("The Common Movie in Recommend List & User:", list(set(Recommend) & set(Input_User_Watched_Movieid)))
    
    return Final_Recommend_Movie

In [40]:
# def User_Based(Input_User, User_Similarity, User_Predictions, Similar_User_Count, Rating_Threshold):
Recommended_Movies = User_Based(67, User_Similarity, User_Prediction,5,1.5)

['Professional, The (1994)']
['Dances with Wolves (1990)']
['Snow White and the Seven Dwarfs (1937)']
['Princess Bride, The (1987)']
['Apocalypse Now (1979)']
['Men in Black (1997)']
["Marvin's Room (1996)"]
The Common Movie in Recommend List & User: []


In [41]:
len(Recommended_Movies)

7